In [280]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import gzip
import pickle
import tensorflow as tf
import pandas as pd
from keras.utils import np_utils
import numpy as np
import itertools
import csv
import random
import math
import os
import keras
from PIL import Image
from keras.layers import Dense
from keras.models import Sequential
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras import optimizers
from sklearn.ensemble import VotingClassifier
from scipy import stats
from statistics import mode 
import scipy
import statistics 

Extract data

In [281]:
TrainingDataFile = 'mnist.pkl.gz'
Data = gzip.open(TrainingDataFile,'rb')
training,validation,testing= pickle.load(Data,encoding='latin1')
trainingData = training[0]
trainingTarget = training[1]
validationData = validation[0]
validationTarget = validation[1]
testingData = testing[0]
testingTarget = testing[1]

Logistic Regression

In [337]:
t = np.zeros(shape=(50000,10))
for x in range(len(trainingData)):
    t[x][trainingTarget[x]] = 1
v = np.zeros(shape=(10000,10))
for x in range(len(validationData)):
    v[x][validationTarget[x]] = 1
te = np.zeros(shape=(10000,10))
for x in range(len(testingData)):
    te[x][testingTarget[x]] = 1

def calculateSoftmax(x):
  return (np.exp(x)/np.sum(np.exp(x)))

def prediction(data,weights):
    return(calculateSoftmax(np.dot(data,weights)))

def calculateErrorFunction(weights,target,data):
    predict = prediction(data,weights)
    error = (-1/len(target))*np.sum(np.multiply(target,np.log(predict)))
    return error

def probability(data,weights):
    return np.argmax(calculateSoftmax((data.T*weights)),axis=1)

W_Now = np.random.randn(trainingData.shape[1],10)*0.05
errorList = []
Val = []
TR = []
Test = []
TrainingAcc = []
ValidationAcc = []
TestingAcc = []
learningRate =0.001
index = 0


def getWeights(W_Now,iteration):
    for i in range(iteration):
        predict = prediction(trainingData,W_Now)
        gradient = np.dot(trainingData.T,predict-t)
        gradient = gradient/len(t)
        W_NEXT = W_Now - (learningRate * gradient)
        W_Now = W_NEXT
    return W_Now
W_Now = getWeights(W_Now,300)
W_Now = np.matrix(W_Now)

Random Forest

In [338]:
randomForest = RandomForestClassifier(n_estimators = 20,n_jobs=1,criterion = 'entropy')
randomForest.fit(trainingData,trainingTarget)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

SVM

In [339]:
svm = SVC(kernel = 'linear',gamma=0.01)
svm.fit(trainingData,trainingTarget)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Neural Network

In [340]:
(MNISTtrainingData,MNISTtrainingTarget),(MNISTtestingData,MNISTtestingTarget) = mnist.load_data()
MNISTtrainingData = MNISTtrainingData.reshape(MNISTtrainingData.shape[0],784)
MNISTtestingData = MNISTtestingData.reshape(MNISTtestingData.shape[0],784)
MNISTtrainingTarget = tf.keras.utils.to_categorical(MNISTtrainingTarget,10)
MNISTtestingTarget = tf.keras.utils.to_categorical(MNISTtestingTarget,10)
Model =Sequential()
Model.add(Dense(units=32,activation='relu',input_shape=(784,)))
Model.add(Dense(units=32,activation='relu',input_shape=(784,)))
Model.add(Dense(units=10,activation='softmax'))
Model.compile(optimizer ='RMSprop',loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = Model.fit(MNISTtrainingData,MNISTtrainingTarget,batch_size= 128,epochs = 30,verbose=False, validation_split= 0.1)

Combine Four Models( From Scratch ) 

In [464]:
correct = 0
from collections import Counter
prediction = []
c = 0
iteration = 100
for y in range(iteration):
    index = np.random.choice(len(testingData), 1, replace=False)
    #print(index)
    testingDataSample = [testingData[i] for i in index]
    testingTargetSample = [testingTarget[i] for i in index]
    randomForestAcc = randomForest.predict(testingDataSample)    #predict the target use random forest
    RF = randomForestAcc
    NNDataarray = np.array(testingDataSample)
    NNTargetarray = np.array(testingTargetSample)
    NNtestingData =  NNDataarray.reshape( NNDataarray.shape[0],784)
    neuralNetworkAcc = Model.predict(NNtestingData,verbose = False)   
    neuralNetworkAcc = list(neuralNetworkAcc)
    NN = []
    NN = np.array(NN)
    for x in range(0,len(neuralNetworkAcc)):
        maxIndex = np.max(neuralNetworkAcc[x])
        NN = np.append(NN,np.where(neuralNetworkAcc[x]==maxIndex))      #predict the target use neural network
    NN = NN.astype(int)                                          
    SVM = svm.predict(testingDataSample)
    LRtestingDataSample = np.array(testingDataSample)
    LR = []
    LR = np.array(LR)
    for x in range(0,len(LRtestingDataSample)):
        LR = np.append(LR ,probability(LRtestingDataSample[x].T,W_Now))    # #predict the target use logistic regression
    LR  = LR.astype(int)
    compareList = []
    compareList.append(RF[x])  
    compareList.append(SVM[x])
    compareList.append(NN[x])
    compareList.append(LR[x])                # append all the result getting from 4 classifiers into the list
    numberElements = Counter(compareList)
    maximum = max(numberElements.values())
    mode = [j for j,z in numberElements.items() if z == maximum]
    prediction = (mode[0])               # pick the most common value into the prediction list, if they have even
    c +=1                                    # number, then pick the first occur common value
    if prediction == testingTargetSample:
        correct+=1

accuracy = (correct/iteration)*100
print("The accuray is:",accuracy,'%')
        


The accuray is: 95.0 %
